# 0. ハンズオンの概要

公式のサンプルを参考に作成した資料になります。公式サンプルではpythonでAutopilotを実行してますが、SageMaker StudioのSageMaker StudioのGUIで実行するよう書き換えています。ハンズオン用のデータの準備やモデルの評価はPythonを実行して行います。


# 1. セットアップ

データを配置するS3を指定します。いい感じのバケットがなければ作成する。

In [ ]:

bucket = "<バケット名>"
prefix = "sagemaker/autopilot-handson"


データを解凍するためunzipをインストールします。

In [ ]:
!apt-get install -y sudo



In [ ]:
!sudo apt-get install -y unzip

# 2. データセットの準備

公式サンプルでも利用されているサンプルをダウンロードします。

In [ ]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip

データを解凍・見てみる。ひょっとしたらエラーになるかも。

解凍

In [ ]:

!unzip -o DKD2e_data_sets.zip

表示

In [ ]:
import pandas as pd

churn = pd.read_csv('Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn

## データについて

電話オペレータ？の対応ごとのユーザーが解約したか・してないかのデータ。今回は「Churn?」の値(True or False)を予測するためAutopilotでモデルを作成する。

トレーニングデータとテストデータにわける。

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(churn, test_size = 0.2, train_size = 0.8, shuffle = True)

トレーニングデータをCSVにして先ほど作成したS3へアップロードします。

In [ ]:

train.to_csv("train.csv", index=False)


In [ ]:
import boto3

s3 = boto3.resource("s3")
s3.meta.client.upload_file("train.csv", bucket, "train.csv")

# 3. Autopilotの実行

Autopilotで学習を実行します。ノートブック上での作業ではなく、SageMaker StudioのGUIでの作業になります。

## ジョブの作成

Welcome画面の ```Create autopilot experiments``` をクリック。Welcome画面を閉じってしまっているなら左上の ```Amazon SageMaker Studio``` をクリックすると開く。

![画像](img/12.PNG)

- ジョブ名を入力。
- 学習データのS3の場所を指定。
    - 上のコードでアップロードしたもの。

![画像](img/13.PNG)

- ```Target attribute name``` は ```Churn?``` 。
- アウトプットの場所を指定。
    - 新規の場所を指定する場合は ```Find S3 bucket``` だとできないので ```Enter S3 bucket location``` で指定。

![画像](img/14.PNG)

- ```Select the machine learning problem type``` はせっかくなので```Auto``` 。
- ```Do you want to run a complete experiment?``` もせっかくなので ```Yes``` 。

![画像](img/15.PNG)

## あとは待つだけ

ジョブが完了するのを待ちます。ただし1時間くらいかかる。

![画像](img/16.PNG)



## ジョブのタブを閉じてしまったら？

```Amazon Experiments Lists``` から 作成したジョブを右クリック。 ```Describe AutoML Job``` で開くことができる。

![画像](img/17.PNG)



## 時間がかかるので今回は。。。

途中でstopしてそこまでで一番よさげなモデルを使います。


# 4. モデルのデプロイ

作成したモデルをデプロイします。

- Autopilotジョブの画面のTrialsを ```Objective``` クリックして並べ替えます。

![画像](img/18.PNG)

- 星がついてるものが一番よさげなモデル。選択して ```Deploy model``` をクリック。

![画像](img/19.PNG)


- モデルのエンドポイント名を入力。インスタンスサイズはm5.largeくらいで。インスタンス数はでもよい。
   - ```Data capture``` は今回はいらない。

![画像](img/20.PNG)

しばらくするとデプロイされる

# モデルで推論

ここからはほぼ公式サンプルの通り。

In [ ]:
import sagemaker

from io import StringIO
from sagemaker.predictor import RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV

# predicterの定義。どのエンドポイント使うか
session = sagemaker.Session()

predictor = RealTimePredictor(
    endpoint="エンドポイント名", 
    sagemaker_session=session,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_CSV)

# 上の方で定義したtestからChurn?列を削除
test_data_inference = test.drop('Churn?', axis=1)

# SageMakerエンドポイントでtestデータの推論を実行
prediction = predictor.predict(test_data_inference.to_csv(sep=',', header=False, index=False)).decode('utf-8')

# 実際の値と推論結果とを比べて精度を出す。
prediction_df = pd.read_csv(StringIO(prediction), header=None)
accuracy = (test_data.reset_index()['Churn?'] == prediction_df[0]).sum() / len(test_data_inference)
print('Accuracy: {}'.format(accuracy))

まあまあ。。。かな。。。？

# エンドポイント削除

エンドポイントは課金され続けるので削除する。



In [ ]:
sm = boto3.client("sagemaker")

sm.delete_endpoint(EndpointName="エンドポイント名")